In [0]:
"""
Amadeus Challenge : Data Scientist Position
Author            : Angeloni Julien
Date Creation     : 10/08/2018

Bonus exercise two - parsing data to JSON format

Data              : bookings.csv
"""

In [0]:
!pip install GeoBases3K

In [0]:
# LIBRARIES
import pandas as pd
from google.colab import drive
from GeoBases import GeoBase
import re

In [2]:
# IMPORT FILES
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
# GLOBAL VARIABLES
BOOKINGS = '/content/gdrive/My Drive/Colab Notebooks/bookings.csv'
GEO_O = GeoBase(data='ori_por', verbose=False)

# WORK USING FUNCTIONS

In [0]:
#METHOD PART

def get_df_cols(csvfilename,cols,separator):
  """
  Method to get a dataframe from a csv file with specified columns
  
  @csvfilename : the name of the file to convert in dataframe
  @cols        : list of string giving columns name to keep
  @separator   : character used to delimit fields in the csv file
  
  @return      : a dataframe
  """
  
  dataframe = pd.read_csv(BOOKINGS, error_bad_lines=False, encoding='UTF8', sep=separator, usecols=cols)
  
  return dataframe


def get_name(IATA_code):
  """
    Function to return the name of the airport linked to IATA_code
    
    @IATA_code : String object which is a IATA_code
   
    @return    : String object which is the name of the airport
  """
  
  #If IATE CODE exists in GEO_O
  try:
    #We found that in the csv file, iata code are followed by some blankspacies
    result = GEO_O.get(IATA_code.replace(" ",""), 'name')
  #Else we just specify that we cannot found the IATA CODE
  except KeyError as e:
    result = "NOT FOUND IATA CODE"
   
  return result
  

def get_airports_arrival_sorted(dataframe):
  """
  Method to print the get arrivals airports in 2013 from searches file
  
  @dataframe : the dataframe containing the data
  
  @return    : a new dataframe
  """
  
  #Created dataframe grouped by 'arr_port' aggregated by sum
  result_dataframe = dataframe.groupby(['arr_port']).sum()
  #Sorted the result in a descending way
  result_dataframe = result_dataframe.sort_values(by=['pax'], ascending=False)
  
  return result_dataframe


def add_airports_name(dataframe):
  """
  Method to add a column in a dataframe containing the full name of airports
  thanks to the IATA CODE
  
  @dataframe : the dataframe to modify
  
  @return    : the dataframe modified
  """
  
  #Reset the index of the dataframe in order to apply a lambda method
  dataframe = dataframe.reset_index()
  
  #Add the column and its values
  dataframe['airport_name'] = dataframe['arr_port'].apply(lambda x: get_name(x))

  return dataframe

def print_top_n_arrival_airport(dataframe,n):
  """
  Method to print the top n of arrival airports in 2013
  
  @dataframe : the preformatted dataframe by columns containing the data
  @n         : the number of airports to show
  """
  
  df = get_airports_arrival_sorted(dataframe)
  df = add_airports_name(df)
  
  print(df.head(n))
  
  return df[0:n]

In [0]:
#PRINT THE TOP TEN
cols = ["arr_port","pax"]

dataframe = get_df_cols(BOOKINGS,cols,'^')

print_top_n_arrival_airport(dataframe,10)

   arr_port      pax                          airport_name
0  LHR       88809.0               London Heathrow Airport
1  MCO       70930.0         Orlando International Airport
2  LAX       70530.0     Los Angeles International Airport
3  LAS       69630.0       Mc Carran International Airport
4  JFK       66270.0  John F Kennedy International Airport
5  CDG       64490.0             Paris - Charles-de-Gaulle
6  BKK       59460.0                          Suvarnabhumi
7  MIA       58150.0           Miami International Airport
8  SFO       58000.0   San Francisco International Airport
9  DXB       55590.0           Dubai International Airport


In [0]:
# CREATE DATAFRAME FROM BOOKINGS_DF_EX GROUPED BY arr_port
BOOKINGS_GROUP_BY_ARR_PORT = BOOKINGS_DF_EX.groupby(['arr_port']).sum()

# JSON PARSING PART

In [0]:
cols = ["arr_port","pax"]

dataframe = get_df_cols(BOOKINGS,cols,'^')

In [6]:
df_to_parse = print_top_n_arrival_airport(dataframe,14)

    arr_port      pax                             airport_name
0   LHR       88809.0                  London Heathrow Airport
1   MCO       70930.0            Orlando International Airport
2   LAX       70530.0        Los Angeles International Airport
3   LAS       69630.0          Mc Carran International Airport
4   JFK       66270.0     John F Kennedy International Airport
5   CDG       64490.0                Paris - Charles-de-Gaulle
6   BKK       59460.0                             Suvarnabhumi
7   MIA       58150.0              Miami International Airport
8   SFO       58000.0      San Francisco International Airport
9   DXB       55590.0              Dubai International Airport
10  ORD       53200.0     Chicago O'Hare International Airport
11  FCO       45120.0      Leonardo da Vinci–Fiumicino Airport
12  IST       43850.0                       Istanbul / Ataturk
13  DFW       42420.0  Dallas/Fort Worth International Airport


In [7]:
json_string = df_to_parse.to_json(orient='split')
json_string

'{"columns":["arr_port","pax","airport_name"],"index":[0,1,2,3,4,5,6,7,8,9,10,11,12,13],"data":[["LHR     ",88809.0,"London Heathrow Airport"],["MCO     ",70930.0,"Orlando International Airport"],["LAX     ",70530.0,"Los Angeles International Airport"],["LAS     ",69630.0,"Mc Carran International Airport"],["JFK     ",66270.0,"John F Kennedy International Airport"],["CDG     ",64490.0,"Paris - Charles-de-Gaulle"],["BKK     ",59460.0,"Suvarnabhumi"],["MIA     ",58150.0,"Miami International Airport"],["SFO     ",58000.0,"San Francisco International Airport"],["DXB     ",55590.0,"Dubai International Airport"],["ORD     ",53200.0,"Chicago O\'Hare International Airport"],["FCO     ",45120.0,"Leonardo da Vinci\\u2013Fiumicino Airport"],["IST     ",43850.0,"Istanbul \\/ Ataturk"],["DFW     ",42420.0,"Dallas\\/Fort Worth International Airport"]]}'